## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## preparation

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import pickle
import cv2
import glob

%matplotlib inline


## First, I'll compute the camera calibration using chessboard images

In [2]:
# calibrate camera using collection of chessboard pictures. 
# Note: 
# 1. calibration will not happen if found coners less than half of the checkboard images 
# 2. Same image size for the correlation image and future image to be undistorted
# images_path: input file name patterns, default value:camera_cal/calibration*.jpg
# corner_num_x: number of corners along the x axis, default value:9
# corner_num_y: number of corners along the y axis, default value:6
# output_path:  output the corner found images for debugging purpose. default value:"camera_cal_output"
# img_size: this corresponds to the picture to be undistorted. default value: None, use the calibration img size
#
# The size of the image, which is passed into the calibrateCamera function, is just the height and width of the image.
# One way to retrieve these values is by retrieving them from the grayscale image shape array gray.shape[::-1]. 
# This returns the image width and height in pixel values like (1280, 960).
# Another way to retrieve the image shape, is to get them directly from the color image by retrieving the first two 
# values in the color image shape array using img.shape[1::-1]. This code snippet asks for just the first two values 
# in the shape array, and reverses them. Note that in our case we are working with a greyscale image, 
# so we only have 2 dimensions (color images have three, height, width, and depth), so this is not necessary.

# Return value - same as cv2.calibrateCamera return
# plus result saved to <output_path>/cam_calibration_result_pickle.p for future use
 

def calibrate_camera (images_path="camera_cal/calibration*.jpg", corner_num_x=9, corner_num_y=6, output_path="camera_cal_output/", img_size=None):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((corner_num_y*corner_num_x,3), np.float32)
    objp[:,:2] = np.mgrid[0:corner_num_x, 0:corner_num_y].T.reshape(-1,2)

    # Make a list of calibration images
    images = glob.glob(images_path)


    objpoints = [] # 3D points in real world space
    imgpoints = [] # 2D points in img plane

    # prepare output file dir
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Step through the list and search for chessboard corners
    corners_found = 0
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (corner_num_x,corner_num_y), None)

        # If found, add object points, image points
        if ret == True:
            corners_found += 1
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, (corner_num_x,corner_num_y), corners, ret)
            write_name = output_path+'corners_found_in_'+os.path.basename(fname)
            cv2.imwrite(write_name, img)
            print ("corners found and saved as:" + write_name)

        else:
            print ("coners not found in: " + fname)
       
    
    if corners_found > idx / 2:
        print ("OK: found cornders in more than half of the input images, carry on with calibration")
        if img_size is None:
            img_size = (img.shape[1], img.shape[0])
            print("img_size is None, use calibration img size: " + str(img_size))
            
            
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
        
        # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
        dist_pickle = {}
        dist_pickle["mtx"] = mtx
        dist_pickle["dist"] = dist
        cam_calibration_result = output_path+"cam_calibration_result_pickle.p"
        pickle.dump( dist_pickle, open( cam_calibration_result, "wb" ) )
        
        print("camera calibration result saved in file " + cam_calibration_result + " for future use" )
    
        return ret, mtx, dist, rvecs, tvecs
        
    else:
        print ("Error: found cornders in less than half of the input images, stopped calibration.")
        return -1, None, None, None, None



##  Note of Applying a distortion correction to raw images.
No wrapping code needed. please use cv2.undistort directly in the future pipeline.
Example code below: 

In [3]:
# Read in the saved camera matrix and distortion coefficients from the pickle dump file
# dist_pickle = pickle.load( open( "wide_dist_pickle.p", "rb" ) )
# mtx = dist_pickle["mtx"]
# dist = dist_pickle["dist"]
# Use the OpenCV undistort() function to remove distortion
# undist = cv2.undistort(img, mtx, dist, None, mtx)

## Then, Use color transforms, gradients, etc., to create a thresholded binary image

In [4]:

# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def corners_unwarp(img, nx, ny, mtx, dist):
    # Use the OpenCV undistort() function to remove distortion
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # Convert undistorted image to grayscale
    gray = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)
    # Search for corners in the grayscaled image
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    if ret == True:
        # If we found corners, draw them! (just for fun)
        cv2.drawChessboardCorners(undist, (nx, ny), corners, ret)
        # Choose offset from image corners to plot detected corners
        # This should be chosen to present the result at the proper aspect ratio
        # My choice of 100 pixels is not exact, but close enough for our purpose here
        offset = 100 # offset for dst points
        # Grab the image shape
        img_size = (gray.shape[1], gray.shape[0])

        # For source points I'm grabbing the outer four detected corners
        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        # For destination points, I'm arbitrarily choosing some points to be
        # a nice fit for displaying our warped result 
        # again, not exact, but close enough for our purposes
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                                     [img_size[0]-offset, img_size[1]-offset], 
                                     [offset, img_size[1]-offset]])
        # Given src and dst points, calculate the perspective transform matrix
        M = cv2.getPerspectiveTransform(src, dst)
        # Warp the image using OpenCV warpPerspective()
        warped = cv2.warpPerspective(undist, M, img_size)

    # Return the resulting image and matrix
    return warped, M




## Pipeline linking all processors

In [5]:
# detect lane from a image file
#




def adv_lane_detection_pipeline(initial_img_path, calibration_mtx, calibration_dist):
    img = cv2.imread(initial_img_path)
    img_size = (img.shape[1], img.shape[0])
    
    # Apply a distortion correction to raw images.
    undist = cv2.undistort(img, calibration_mtx, calibration_dist, None, calibration_mtx)

    # Use color transforms, gradients, etc., to create a thresholded binary image.
    
    ## Convert undistorted image to grayscale
    #gray = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)
    

    
    # Apply a perspective transform to rectify binary image ("birds-eye view").
    
    
    # Detect lane pixels and fit to find the lane boundary.
    
    
    # Determine the curvature of the lane and vehicle position with respect to center.
    
    
    # Warp the detected lane boundaries back onto the original image.
    
    
    # Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

    result = undist

    return result




In [6]:
# Calibrate camera

ret, mtx, dist, rvecs, tvecs = calibrate_camera()
print(ret)

if ret < 0:
    print ("Error: calibration camera failed.")
    exit 

# iterate through every input file
INPUT_DIR = "test_images/"
OUTPUT_DIR = "test_images_output/"
os.makedirs(os.path.dirname(OUTPUT_DIR), exist_ok=True)

for filename in os.listdir(INPUT_DIR):    
    result_img = adv_lane_detection_pipeline (INPUT_DIR+filename, mtx, dist)
    cv2.imwrite(OUTPUT_DIR+filename, result_img)

coners not found in: camera_cal/calibration5.jpg
coners not found in: camera_cal/calibration4.jpg
corners found and saved as:camera_cal_output/corners_found_in_calibration6.jpg
corners found and saved as:camera_cal_output/corners_found_in_calibration7.jpg
corners found and saved as:camera_cal_output/corners_found_in_calibration3.jpg
corners found and saved as:camera_cal_output/corners_found_in_calibration2.jpg
coners not found in: camera_cal/calibration1.jpg
corners found and saved as:camera_cal_output/corners_found_in_calibration20.jpg
corners found and saved as:camera_cal_output/corners_found_in_calibration19.jpg
corners found and saved as:camera_cal_output/corners_found_in_calibration18.jpg
corners found and saved as:camera_cal_output/corners_found_in_calibration15.jpg
corners found and saved as:camera_cal_output/corners_found_in_calibration14.jpg
corners found and saved as:camera_cal_output/corners_found_in_calibration16.jpg
corners found and saved as:camera_cal_output/corners_foun

In [7]:

#top_down, perspective_M = corners_unwarp(img, nx, ny, mtx, dist)
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()
#ax1.imshow(img)
#ax1.set_title('Original Image', fontsize=50)
#ax2.imshow(top_down)
#ax2.set_title('Undistorted and Warped Image', fontsize=50)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

